In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, ConcatDataset

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import json

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from segmentation_models_pytorch.utils.metrics import IoU, Fscore, Accuracy

In [3]:
import matplotlib.pyplot as plt

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, 'gray')
    plt.show()

In [4]:
import random

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print(np.random.rand(5), torch.randn(5))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ] tensor([ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845])


# Settings

In [5]:
root = os.getcwd()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BATCH = 4

GEN_IN_CHANNELS = 1
GEN_N_CLASSES = 1

DIS_IN_CHANNELS = 1
DIS_N_CLASSES = 2

E_IN_CHANNELS = 1
E_N_CLASSES = 2

EPOCH = 200

### Model Settings

#### encoder

In [6]:
ENCODER = "resnet152"
ENCODER_WEIGHT = None

#### decoder

In [7]:
DECODER_ATT = "scse"

#### head

In [8]:
GEN_ACT = "sigmoid"
DIS_ACT = "softmax"
E_ACT = "softmax"

### Optimizer Settings

In [9]:
GEN_OPTIM_NAME = "adam"
GEN_init_lr = 1e-4
GEN_momentum = 0.9

In [10]:
DIS_OPTIM_NAME = "adam"
DIS_init_lr = 1e-3
DIS_momentum = 0.9
DIS_scheduler = "cosineAnnWarm"

In [11]:
E_OPTIM_NAME = "adam"
E_init_lr = 1e-3
E_momentum = 0.9
E_scheduler = "cosineAnnWarm"

# Epochs

In [12]:
from codes.losses import SSIMLoss
from codes.losses import MAELoss
from pytorch_msssim import ssim
from codes.metrics import PSNR, SNR, ContourEval
from codes.activation import Activation
from codes.utils import hu_clip_tensor
from codes.losses import PerceptualLoss
from kornia.filters.sobel import Sobel

In [13]:
def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad=Fasle for all the networks to avoid unnecessary computations
    Parameters:
    nets (network list)   -- a list of networks
    requires_grad (bool)  -- whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad

In [14]:
def replace_relu_to_leakyReLU(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, nn.LeakyReLU())
        else:
            replace_relu_to_leakyReLU(child)

In [15]:
def replace_bn_to_instanceNorm(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.BatchNorm2d):
            bn = getattr(model, child_name)
            setattr(model, child_name, nn.InstanceNorm2d(bn.num_features))
        else:
            replace_bn_to_instanceNorm(child)

In [16]:
perceptual_ext = PerceptualLoss()

In [17]:
sobel_filter = Sobel().to(device)

In [18]:
@torch.no_grad()
def eval_epoch(generator, tf_discriminator, edge_discriminator, dataloader, device):
  
    # change mode to train and move to current device
    generator = generator.eval().to(device)
    tf_discriminator = tf_discriminator.eval().to(device)
    edge_discriminator = edge_discriminator.eval().to(device)
    
    ssim_ = []
    psnr_ = []
    mae_ = []
    snr_ = []
    
    air_ = []
    bone_ = []
    cont_ = []
 
    tf_acc = []
    edge_acc = []
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = generator(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        total = x.size()[0]
       ############################
        # (1) Eval True/Fake D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################        
        y_cls = tf_discriminator(y)
        for _y_cls in y_cls:
            _y_cls = Activation(name=DIS_ACT)(_y_cls)
            _, _y_cls = torch.max(_y_cls.data, 1)
            correct = (_y_cls == torch.zeros(total, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]        
        
        y_pr_cls = tf_discriminator(y_pr)
        for _y_pr_cls in y_pr_cls:
            _y_pr_cls = Activation(name=DIS_ACT)(_y_pr_cls)
            _, _y_pr_cls = torch.max(_y_pr_cls.data, 1)
            correct = (_y_pr_cls == torch.ones(total, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]
        
       ############################
        # (1.1) Eval Edge D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################  
        edge_y_cls = edge_discriminator(edge_y)
        for _edge_y_cls in edge_y_cls:
            _edge_y_cls = Activation(name=DIS_ACT)(_edge_y_cls)
            _, _edge_y_cls = torch.max(_edge_y_cls.data, 1)
            correct = (_edge_y_cls == torch.zeros(total, dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]          
        
        edge_pr_cls = edge_discriminator(edge_y_pr)
        for _edge_pr_cls in edge_pr_cls:
            _edge_pr_cls = Activation(name=DIS_ACT)(_edge_pr_cls)
            _, _edge_pr_cls = torch.max(_edge_pr_cls.data, 1)
            correct = (_edge_pr_cls == torch.ones(total, dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]        
        
        ############################
        # (2) Eval G network: maximize log(D(G(z)))
        ###########################
        
        # main
        _ssim = ssim(y, y_pr, data_range=1.0, size_average=True) # return (N,)
        _psnr = PSNR()(y_pr, y, 1.0)
        _snr = SNR()(y_pr, y)
        _mae = MAELoss()(y_pr.float(), y.float())

        ssim_ += [_ssim.item()]
        psnr_ += [_psnr.item()]
        mae_ += [_mae.item()]
        snr_ += [_snr.item()]
        
        # auxilary
        _min = -500
        _max = 500
        air_window = (-500, -499)
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(air_pr, air_x)
        air_ += [dice.item()]
        
        bone_window = (255, 256)
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(bone_pr, bone_x)
        bone_ += [dice.item()]

        for b in range(x.size()[0]):
            cont = ContourEval()(y_pr[b, :, :, :], x[b, :, :, :])
            cont_ += [cont.item()]

    return  sum(ssim_)/len(ssim_), sum(psnr_)/len(psnr_), sum(snr_)/len(snr_), sum(mae_)/len(mae_), \
                    sum(air_)/len(air_), sum(bone_)/len(bone_), sum(cont_)/len(cont_), sum(tf_acc)/len(tf_acc), sum(edge_acc)/len(edge_acc)

In [19]:
@torch.no_grad()
def test_epoch(iid, model, dataloader, device, save=False, path=None):
  
    # change mode to train and move to current device
    model = model.eval().to(device)
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = model(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        _min = -500
        _max = 500
        air_window = (-500, -499)
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        
        bone_window = (255, 256)
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)

        for b in range(x.shape[0]):           
            tmp_y = y[b, :, :, :].unsqueeze(0)
            tmp_y_pr = y_pr[b, :, :, :].unsqueeze(0)
            tmp_air = air_x[b, :, :, :].unsqueeze(0)
            tmp_air_pr = air_pr[b, :, :, :].unsqueeze(0)
            tmp_bone = bone_x[b, :, :, :].unsqueeze(0)
            tmp_bone_pr = bone_pr[b, :, :, :].unsqueeze(0)
            
            _ssim = ssim(tmp_y, tmp_y_pr, data_range=1.0, size_average=True) # return (N,)
            _psnr = PSNR()(tmp_y_pr, tmp_y, 1.0)
            _snr = SNR()(tmp_y_pr, tmp_y)
            _mae = MAELoss()(tmp_y_pr.float(), tmp_y.float())
            _air = Fscore()(tmp_air_pr, tmp_air)
            _bone = Fscore()(tmp_bone_pr, tmp_bone)
            _cont = ContourEval()(tmp_y_pr, x[b, :, :, :])
            
            scores = {
                "ssim score": _ssim.item(),
                "psnr score": _psnr.item(),
                "snr score": _snr.item(),
                "mae error": _mae.item(),
                "air dice score": _air.item(),
                "bone dice score": _bone.item(),
                "contour dice score": _cont.item()
            }
            
            __cbct = (x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct = (tmp_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_pred = (tmp_y_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_masked = __ct_pred * tmp_air.squeeze().cpu().numpy()
            __air = (tmp_air.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __air_pr = (tmp_air_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone = (tmp_bone.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone_pr = (tmp_bone_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_pr = (edge_y_pr[b, :].squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_x = (edge_x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_y = (edge_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            
#             visualize(
#                 cbct = __cbct,
#                 ct_y = __ct,
#                 ct_pred = __ct_pred,
#                 edge_pred = __edge_pr,
#                 edge_x = __edge_x,
#                 edge_y = __edge_y,
#                 air = __air,
#                 air_pr = __air_pr,
#                 bone = __bone,
#                 bone_pr = __bone_pr
#             )
            
            if save:
                path_dir = os.path.join(path, "file_{}".format(iid))
                try:
                    os.mkdir(path_dir)
                except FileExistsError:
                    pass
                
                cv2.imwrite(os.path.join(path_dir, "cbct.jpg"), __cbct)
                cv2.imwrite(os.path.join(path_dir, "ct.jpg"), __ct)
                cv2.imwrite(os.path.join(path_dir, "ct_pred.jpg"), __ct_pred)
                cv2.imwrite(os.path.join(path_dir, "air.jpg"), __air)
                cv2.imwrite(os.path.join(path_dir, "air_pred.jpg"), __air_pr)
                cv2.imwrite(os.path.join(path_dir, "bone.jpg"), __bone)
                cv2.imwrite(os.path.join(path_dir, "bone_pred.jpg"), __bone_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_pred.jpg"), __edge_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_cbct.jpg"), __edge_x)
                cv2.imwrite(os.path.join(path_dir, "edge_ct.jpg"), __edge_y)
                with open(os.path.join(path_dir, "scores.txt"), "w") as file:
                    file.write(json.dumps(scores))                
            iid += 1

# Discriminator

In [20]:
from codes.activation import Activation
import torchvision.models as models
from codes.losses import MultiScaleHeads

In [21]:
class Discriminator(nn.Module):
    def __init__(self, in_channel=1, n_classes=2, activation=None):
        
        super(Discriminator, self).__init__()
        
        encoder = models.resnet18()
        encoder.conv1 = nn.Conv2d(in_channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        encoder.fc = nn.Linear(in_features=512, out_features=n_classes)
        self.encoder = encoder
        self.activation = Activation(name=activation)
        
        
    def forward(self, x):
        x = self.encoder(x)
        return [self.activation(x)]

# Generator

In [22]:
import segmentation_models_pytorch as smp
from codes.decoder import UnetDecoder
from segmentation_models_pytorch.base.heads import SegmentationHead

In [23]:
class Generator(nn.Module):
    def __init__(self, encoder_name, encoder_weights, in_channels, classes, attention_type):
        
        super(Generator, self).__init__()
        
        self.encoder = smp.Unet(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes).encoder
        self.decoder = UnetDecoder(
                            encoder_channels=[in_channels, 0, 0, 0, 0, 2048],
                            decoder_channels=[512, 256, 128, 64, 16],
                            n_blocks=5,
                            use_batchnorm=True,
                            attention_type=attention_type)
        self.head =  SegmentationHead(
                        in_channels=16,
                        out_channels=classes,
                        activation=None)
        
        
    def forward(self, x):
        x = self.encoder(x)[-1]
        x = self.decoder(*(None, None, None, None, x))
        x = self.head(x)
        
        return x

# Read Data

In [24]:
import glob
from codes.dataset import Dataset, DicomDataset, DicomsDataset
import codes.augmentation as aug

In [25]:
# run_name = wandb.run.name
run_name = "zesty-energy-226"

In [26]:
checkpoint = torch.load(os.path.join("weight-gan", "{}.pth".format(run_name)))

In [27]:
model = smp.Unet(encoder_name=ENCODER, 
                  encoder_weights=ENCODER_WEIGHT, 
                  in_channels=GEN_IN_CHANNELS, 
                  classes=GEN_N_CLASSES,
                 decoder_attention_type=DECODER_ATT)
replace_relu_to_leakyReLU(model)
replace_bn_to_instanceNorm(model)
model.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [28]:
tf_discriminator = Discriminator(in_channel=DIS_IN_CHANNELS, n_classes=DIS_N_CLASSES, activation=DIS_ACT)
replace_relu_to_leakyReLU(tf_discriminator)
replace_bn_to_instanceNorm(tf_discriminator)
tf_discriminator.load_state_dict(checkpoint["tf_discriminator"])

<All keys matched successfully>

In [29]:
edge_discriminator = Discriminator(in_channel=E_IN_CHANNELS, n_classes=E_N_CLASSES, activation=E_ACT)
replace_relu_to_leakyReLU(edge_discriminator)
replace_bn_to_instanceNorm(edge_discriminator)
edge_discriminator.load_state_dict(checkpoint["edge_discriminator"])

<All keys matched successfully>

# Pelvic

## Pelvic Test CBCT

In [30]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [31]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [32]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct"))
except FileExistsError:
    pass

In [33]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=False, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id)) 
    iid += len(scans)

28it [00:02, 11.99it/s]
27it [00:01, 20.03it/s]
26it [00:01, 19.52it/s]
28it [00:01, 19.58it/s]
27it [00:01, 19.93it/s]
28it [00:01, 20.20it/s]
27it [00:01, 19.76it/s]


In [34]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=False, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:09, 19.40it/s]


(0.7936573424888531,
 21.378059836582363,
 12.382331863123708,
 0.04077265499383991,
 0.9952817647852372,
 0.8940261644287292,
 0.3742317964971378,
 0.5,
 0.4973821989528796)

## Pelvic Test CT

In [35]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [36]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [37]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct"))
except FileExistsError:
    pass

In [38]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=False, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id)) 
    iid += len(scans)

28it [00:01, 19.01it/s]
27it [00:01, 19.68it/s]
26it [00:01, 19.60it/s]
28it [00:01, 20.41it/s]
27it [00:01, 20.18it/s]
28it [00:01, 19.47it/s]
27it [00:01, 19.60it/s]


In [39]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=False, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:09, 19.98it/s]


(0.8518316761361366,
 23.91654667679552,
 14.920818623447918,
 0.03208012481010397,
 0.9936838775449286,
 0.8846223849755032,
 0.37948852020720536,
 0.5,
 0.4973821989528796)

## Pelvic L1 CBCT

In [40]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [41]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [42]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct"))
except FileExistsError:
    pass

In [43]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=False, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id)) 
    iid += len(scans)

29it [00:01, 20.27it/s]
26it [00:01, 19.07it/s]
28it [00:01, 19.96it/s]
28it [00:01, 19.57it/s]
27it [00:01, 19.68it/s]
26it [00:01, 18.77it/s]
27it [00:01, 19.74it/s]
30it [00:01, 20.10it/s]
28it [00:01, 19.87it/s]
29it [00:01, 19.46it/s]
29it [00:01, 20.31it/s]
27it [00:01, 19.35it/s]
28it [00:01, 19.86it/s]
26it [00:01, 19.74it/s]
28it [00:01, 19.52it/s]
28it [00:01, 19.32it/s]
27it [00:01, 19.45it/s]
26it [00:01, 19.33it/s]
27it [00:01, 19.83it/s]
29it [00:01, 20.01it/s]
30it [00:01, 19.67it/s]
32it [00:01, 19.81it/s]
28it [00:01, 19.46it/s]
28it [00:01, 19.72it/s]
15it [00:00, 18.22it/s]
28it [00:01, 19.69it/s]
26it [00:01, 18.98it/s]
28it [00:01, 19.53it/s]
26it [00:01, 18.96it/s]
27it [00:01, 20.05it/s]
36it [00:01, 19.93it/s]
28it [00:01, 19.18it/s]
27it [00:01, 18.94it/s]
28it [00:01, 20.18it/s]
30it [00:01, 19.84it/s]
27it [00:01, 19.52it/s]
26it [00:01, 19.02it/s]
29it [00:01, 20.01it/s]
27it [00:01, 19.55it/s]
27it [00:01, 19.19it/s]
28it [00:01, 19.98it/s]
26it [00:01, 19.

In [44]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=False, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [01:11, 20.48it/s]


(0.7884765386008693,
 20.8448360341006,
 11.64381832560408,
 0.04457343223727805,
 0.9933169951520787,
 0.8609750589766147,
 0.4027254766124988,
 0.5,
 0.4962251201098147)

## Pelvic L1 CT

In [45]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [46]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [47]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct"))
except FileExistsError:
    pass

In [48]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=False, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id)) 
    iid += len(scans)

29it [00:01, 19.58it/s]
26it [00:01, 18.70it/s]
28it [00:01, 19.43it/s]
28it [00:01, 19.44it/s]
27it [00:01, 19.22it/s]
26it [00:01, 19.27it/s]
27it [00:01, 19.21it/s]
30it [00:01, 19.72it/s]
28it [00:01, 19.45it/s]
29it [00:01, 19.39it/s]
29it [00:01, 20.01it/s]
27it [00:01, 18.85it/s]
28it [00:01, 18.76it/s]
26it [00:01, 19.18it/s]
28it [00:01, 19.38it/s]
28it [00:01, 19.30it/s]
27it [00:01, 19.15it/s]
26it [00:01, 18.81it/s]
27it [00:01, 19.68it/s]
29it [00:01, 19.31it/s]
30it [00:01, 18.71it/s]
32it [00:01, 19.93it/s]
28it [00:01, 19.56it/s]
28it [00:01, 18.71it/s]
15it [00:00, 17.34it/s]
28it [00:01, 17.56it/s]
26it [00:01, 18.43it/s]
28it [00:01, 19.39it/s]
26it [00:01, 19.12it/s]
27it [00:01, 19.15it/s]
36it [00:01, 19.17it/s]
28it [00:01, 18.94it/s]
27it [00:01, 18.89it/s]
28it [00:01, 20.06it/s]
30it [00:01, 19.86it/s]
27it [00:01, 19.01it/s]
26it [00:01, 18.98it/s]
29it [00:01, 18.97it/s]
27it [00:01, 19.23it/s]
27it [00:01, 19.34it/s]
28it [00:01, 18.94it/s]
26it [00:01, 19.

In [49]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=False, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [01:12, 20.23it/s]


(0.8536144467192148,
 23.705038291915443,
 14.504020677172491,
 0.034144915837229485,
 0.994012916210159,
 0.8613707957771218,
 0.45941068656355666,
 0.5,
 0.4876458476321208)

## Pelvic L2 CBCT

In [50]:
test_case_path = 'L2_pelvic_processed/reg_pelvic_l2/*_*'
paths = sorted(glob.glob(test_case_path))

In [51]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [52]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_cbct"))
except FileExistsError:
    pass

In [53]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=False, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_cbct", patient_id)) 
    iid += len(scans)

26it [00:01, 18.80it/s]
26it [00:01, 17.19it/s]
26it [00:01, 18.74it/s]
27it [00:01, 18.58it/s]
31it [00:01, 19.07it/s]
27it [00:01, 18.57it/s]
27it [00:01, 18.74it/s]
26it [00:01, 17.92it/s]
28it [00:01, 18.35it/s]
28it [00:01, 19.00it/s]
27it [00:01, 18.75it/s]
27it [00:01, 18.80it/s]
26it [00:01, 18.37it/s]
26it [00:01, 18.78it/s]
26it [00:01, 18.37it/s]
26it [00:01, 18.51it/s]
26it [00:01, 18.80it/s]
26it [00:01, 18.85it/s]
27it [00:01, 18.33it/s]
26it [00:01, 17.87it/s]
26it [00:01, 18.43it/s]
28it [00:01, 17.98it/s]
28it [00:01, 19.02it/s]
29it [00:01, 17.92it/s]
28it [00:01, 19.09it/s]
26it [00:01, 18.98it/s]
27it [00:01, 17.87it/s]
26it [00:01, 18.51it/s]
28it [00:01, 18.84it/s]
30it [00:01, 18.55it/s]
27it [00:01, 18.59it/s]
29it [00:01, 18.66it/s]
27it [00:01, 18.88it/s]
27it [00:01, 18.63it/s]
26it [00:01, 18.43it/s]
26it [00:01, 18.93it/s]
26it [00:01, 18.44it/s]
26it [00:01, 17.98it/s]
27it [00:01, 18.74it/s]
26it [00:01, 17.43it/s]
26it [00:01, 19.02it/s]
26it [00:01, 18.

In [54]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=False, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

3064it [04:39, 10.95it/s]


(0.7806857549451661,
 19.964970321630375,
 10.380122222881715,
 0.04958785807120543,
 0.9899601340041939,
 0.8107263032897394,
 0.41741360115359444,
 0.5,
 0.49363577023498695)

## Pelvic L2 CT

In [55]:
test_case_path = 'L2_pelvic_processed/reg_pelvic_l2/*_*'
paths = sorted(glob.glob(test_case_path))

In [56]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [57]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct"))
except FileExistsError:
    pass

In [58]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=False, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id)) 
    iid += len(scans)

26it [00:02, 12.09it/s]
26it [00:02, 11.09it/s]
26it [00:02, 12.08it/s]
27it [00:02, 12.06it/s]
31it [00:02, 11.99it/s]
27it [00:02, 11.46it/s]
27it [00:02, 11.87it/s]
26it [00:02, 11.78it/s]
28it [00:02, 11.83it/s]
28it [00:02, 12.18it/s]
27it [00:02, 11.51it/s]
27it [00:02, 11.33it/s]
26it [00:02, 11.20it/s]
26it [00:02, 11.21it/s]
26it [00:02, 11.61it/s]
26it [00:02, 11.62it/s]
26it [00:02, 11.25it/s]
26it [00:02, 11.38it/s]
27it [00:03,  7.11it/s]
26it [00:02,  9.55it/s]
26it [00:02,  9.79it/s]
28it [00:02,  9.87it/s]
28it [00:02,  9.92it/s]
29it [00:02, 10.48it/s]
28it [00:02,  9.82it/s]
26it [00:02, 12.92it/s]
27it [00:02,  9.98it/s]
26it [00:02,  9.96it/s]
28it [00:02,  9.94it/s]
30it [00:02, 10.11it/s]
27it [00:02, 10.07it/s]
29it [00:02, 10.17it/s]
27it [00:02, 10.31it/s]
27it [00:02,  9.10it/s]
26it [00:02, 12.39it/s]
26it [00:02, 11.69it/s]
26it [00:02, 12.38it/s]
26it [00:02, 11.74it/s]
27it [00:02, 11.96it/s]
26it [00:02, 11.81it/s]
26it [00:02, 12.08it/s]
26it [00:02, 12.

In [59]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=False, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

3064it [04:36, 11.07it/s]


(0.8658822867678911,
 23.3832120061232,
 13.798363827071675,
 0.036212776326781076,
 0.9899031124135431,
 0.8538145466823579,
 0.43570569514832674,
 0.5,
 0.49510443864229764)

# Abdomen

In [60]:
test_case_path = 'abdomen/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Abdomen on CBCT

In [61]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [62]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct"))
except FileExistsError:
    pass

In [63]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=False, position="abdomen")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id)) 
    iid += len(scans)

28it [00:02, 11.27it/s]
78it [00:08,  9.67it/s]
50it [00:04, 10.42it/s]
29it [00:02,  9.91it/s]
50it [00:04, 10.05it/s]
71it [00:06, 10.29it/s]
39it [00:04,  8.54it/s]


In [64]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=False, position="abdomen")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

345it [00:34, 10.02it/s]


(0.7126143671464229,
 18.248797076681385,
 8.779438170142797,
 0.06492403624174388,
 0.9865677381822555,
 0.7100269218431963,
 0.3507830173209094,
 0.5,
 0.46956521739130436)

## Abdomen on CT

In [65]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [66]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct"))
except FileExistsError:
    pass

In [67]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=False, position="abdomen")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id)) 
    iid += len(scans)

28it [00:02, 11.33it/s]
78it [00:06, 12.03it/s]
50it [00:04, 12.23it/s]
29it [00:02, 11.67it/s]
50it [00:04, 11.98it/s]
71it [00:05, 11.84it/s]
39it [00:03, 11.78it/s]


In [68]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=False, position="abdomen")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

345it [00:32, 10.65it/s]


(0.819454444491345,
 21.985584308790123,
 12.516225248143293,
 0.045593803262580994,
 0.9867614800297639,
 0.7334432020178642,
 0.42461754308230637,
 0.5,
 0.5101449275362319)

# Chest

In [69]:
test_case_path = 'chest/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Chest on CBCT

In [70]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [71]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct"))
except FileExistsError:
    pass

In [72]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=False, position="chest")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id)) 
    iid += len(scans)

49it [00:04,  9.96it/s]
39it [00:03,  9.77it/s]
40it [00:03, 11.58it/s]
29it [00:03,  8.85it/s]
35it [00:02, 13.45it/s]
37it [00:03,  9.89it/s]
34it [00:03,  9.54it/s]


In [73]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=False, position="chest")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [00:26,  9.97it/s]


(0.7529509164987861,
 18.60664775616316,
 8.158777048379296,
 0.05839493935892337,
 0.916451679849062,
 0.7469037942557132,
 0.3769115421695874,
 0.5,
 0.47338403041825095)

## Chest on CT

In [74]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [75]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct"))
except FileExistsError:
    pass

In [76]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=False, position="chest")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_ct", patient_id)) 
    iid += len(scans)

49it [00:04, 11.81it/s]
39it [00:03, 11.45it/s]
40it [00:03, 11.30it/s]
29it [00:02, 11.27it/s]
35it [00:03, 11.20it/s]
37it [00:03, 11.69it/s]
34it [00:03, 10.80it/s]


In [77]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=False, position="chest")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [00:22, 11.53it/s]


(0.8568198112027273,
 21.885577473803618,
 11.437706737010652,
 0.04347181296818837,
 0.9234351363369342,
 0.7230003543026112,
 0.3929452127648048,
 0.5,
 0.4600760456273764)

# Headneck

In [78]:
test_case_path = 'headneck/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Headneck on CBCT

In [79]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [80]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct"))
except FileExistsError:
    pass

In [81]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=False, position="headneck")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id)) 
    iid += len(scans)

58it [00:04, 12.05it/s]
55it [00:05, 10.52it/s]
55it [00:05, 10.57it/s]
56it [00:04, 11.85it/s]
56it [00:05, 10.72it/s]
55it [00:05, 10.81it/s]
56it [00:05, 10.70it/s]


In [82]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=False, position="headneck")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [00:34, 11.18it/s]


(0.7121929184859975,
 18.128501775014737,
 4.175949064362079,
 0.05055874330765756,
 0.2462905510920413,
 0.6220725531564116,
 0.00041094098587494266,
 0.5,
 0.6739130434782609)

## Headneck on CT

In [83]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [84]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct"))
except FileExistsError:
    pass

In [85]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=False, position="headneck")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_ct", patient_id)) 
    iid += len(scans)

58it [00:04, 12.31it/s]
55it [00:04, 12.03it/s]
55it [00:04, 12.00it/s]
56it [00:04, 12.20it/s]
56it [00:04, 12.12it/s]
55it [00:04, 12.04it/s]
56it [00:04, 12.61it/s]


In [86]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=False, position="headneck")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [00:35, 11.17it/s]


(0.7763209022067087,
 19.38371893451037,
 5.431166150619078,
 0.04622843871107492,
 0.32780775120755856,
 0.6393413724460104,
 0.01889910017455498,
 0.5,
 0.489769820971867)